### 1. CROP TILES

We previously exported each country/year combination in our `drive`.  

- When downloading files from browser,  
google splits them into max-sized chunks that we will have to unpack.  
- Then, we will crop tiles given the occurences' locations.

In [1]:
import glob
import os 

import rasterio as rio
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import xarray as xr 
from shapely import Point, Polygon
import matplotlib.pyplot as plt
import zipfile

In [2]:
DATA_DIR=os.path.join('..','data','landsat_7','')
CSV_PATH=os.path.join('..','data','wealth_index.csv')
DOWNLOAD_DIR=os.path.expanduser('./')              # EX: '~/Downloads/'

0. OPEN AND UPDATE OUR CSV

In [3]:
# UPDATE POINT COORDINATES & CRS
# WE NOW WORK IN METERS WITH EPSG:3857
csv=pd.read_csv(CSV_PATH)
gdf = gpd.GeoDataFrame(csv, geometry=gpd.points_from_xy(csv.lon, csv.lat))
gdf.set_crs(epsg="4326", inplace=True)
gdf = gdf.to_crs('EPSG:3857')
gdf.to_csv(CSV_PATH,index=False)
gdf.head()

,country,year,cluster,lat,lon,households,wealthpooled,geometry,filename,bounding_box
0,angola,2011,1,-12.350257,13.534922,36,2.312757,POINT (1506700.586 -1385596.068),-12_3_13_53.tif,POLYGON ((1503340.58557273 -1388956.0684884773...
1,angola,2011,2,-12.360865,13.551494,32,2.010293,POINT (1508545.372 -1386804.913),-12_3_13_55.tif,POLYGON ((1505185.372017885 -1390164.913024517...
2,angola,2011,3,-12.613421,13.413085,36,0.877744,POINT (1493137.790 -1415600.608),-12_6_13_41.tif,POLYGON ((1489777.790366379 -1418960.607574332...
3,angola,2011,4,-12.581454,13.397711,35,1.066994,POINT (1491426.344 -1411954.259),-12_5_13_39.tif,POLYGON ((1488066.3440705661 -1415314.25888946...
4,angola,2011,5,-12.578135,13.418748,37,1.750153,POINT (1493768.184 -1411575.617),-12_5_13_41.tif,POLYGON ((1490408.1835246533 -1414935.61727971...


1. Unzip files and place them in `DATA_FOLDER`

In [4]:
# GLOB FILES TO UNZIP
files_to_unzip=[]
variable = 'landsat'
files_to_unzip+=glob.glob(os.path.join(DOWNLOAD_DIR,"*"+variable+"*.zip"))

In [ ]:
# UNZIP 
for file in files_to_unzip:
    with zipfile.ZipFile(file) as zip:
        for zip_info in zip.infolist():
            # WE IGNORE FOLDERS ARCHITECTURE
            # AND ONLY RECOVER ACTUAL FILES
            if zip_info.is_dir():
                continue
            zip_info.filename = os.path.basename(zip_info.filename)
            zip.extract(zip_info, DATA_DIR)

2. Crop occurences given the corresponding country-year chunked files

In [5]:
# GLOB FILES PER COUNTRY-YEAR PAIR
files=dict()
for year in csv.year.unique():
    files[year]=dict()
    for country in csv.country.unique():
        files[year][country]=[]
        files[year][country]=glob.glob(os.path.join(DATA_DIR,str(country)+"*"+str(year)+"*.tif"))
files

{2011: {'angola': [],
  'benin': [],
  'burkina_faso': [],
  'cameroon': [],
  'cote_d_ivoire': [],
  'democratic_republic_of_congo': [],
  'ethiopia': [],
  'ghana': [],
  'guinea': [],
  'kenya': [],
  'lesotho': [],
  'madagascar': [],
  'malawi': [],
  'mali': [],
  'mozambique': [],
  'nigeria': [],
  'rwanda': [],
  'senegal': [],
  'sierra_leone': [],
  'tanzania': [],
  'togo': [],
  'uganda': [],
  'zambia': [],
  'zimbabwe': []},
 2015: {'angola': [],
  'benin': [],
  'burkina_faso': [],
  'cameroon': [],
  'cote_d_ivoire': [],
  'democratic_republic_of_congo': [],
  'ethiopia': [],
  'ghana': [],
  'guinea': [],
  'kenya': [],
  'lesotho': [],
  'madagascar': [],
  'malawi': [],
  'mali': [],
  'mozambique': [],
  'nigeria': [],
  'rwanda': [],
  'senegal': [],
  'sierra_leone': [],
  'tanzania': [],
  'togo': [],
  'uganda': [],
  'zambia': [],
  'zimbabwe': []},
 2012: {'angola': [],
  'benin': [],
  'burkina_faso': [],
  'cameroon': [],
  'cote_d_ivoire': [],
  'democrati

In [9]:
def open_tif(file):
    return rxr.open_rasterio(file, masked=True)

def get_bb_from_point(point, margin=112, scale=30):
    """Returns bounding box coordinates from a Point.

    Args:
        point (Geometry.Point): Location
        margin (int, optional): Number of pixels from center to box border . Defaults to 112. (total size = 224)
        scale (int, optional): Number of meter per pixel. Defaults to 30 (landsat scale)

    Returns:
        Geometry.Polygon: Bounding Box
    """
    minx=point.x-margin*scale
    miny=point.y-margin*scale
    maxx=point.x+margin*scale
    maxy=point.y+margin*scale
    return Polygon([
        Point(minx, miny), 
        Point(maxx, miny), 
        Point(maxx, maxy), 
        Point(minx, maxy), 
        Point(minx, miny)
    ])

def clip_geometry(point, tif, margin=112, scale=30):
    try :
        tile = tif.rio.clip_box(
                minx=point.x-margin*scale,
                miny=point.y-margin*scale,
                maxx=point.x+margin*scale,
                maxy=point.y+margin*scale
            )
    except rxr.exceptions.NoDataInBounds:
        return None
    return tile

def write_tile(tile, name, path=os.path.join(DATA_DIR,'')):
    tile.rio.to_raster(path+name+'.tif')
    return 

def add_bb_coords(gdf, margin=112, scale=30):
    gdf['bounding_box'] = gdf['geometry'].map( lambda point: get_bb_from_point(point,margin,scale) )
    return gdf

In [7]:
gdf = add_bb_coords(gdf)
gdf.to_csv(CSV_PATH,index=False)
gdf.head()

,country,year,cluster,lat,lon,households,wealthpooled,geometry,filename,bounding_box
0,angola,2011,1,-12.350257,13.534922,36,2.312757,POINT (1506700.586 -1385596.068),-12_3_13_53.tif,"POLYGON ((1503340.586 -1388956.068, 1510060.58..."
1,angola,2011,2,-12.360865,13.551494,32,2.010293,POINT (1508545.372 -1386804.913),-12_3_13_55.tif,"POLYGON ((1505185.372 -1390164.913, 1511905.37..."
2,angola,2011,3,-12.613421,13.413085,36,0.877744,POINT (1493137.790 -1415600.608),-12_6_13_41.tif,"POLYGON ((1489777.790 -1418960.608, 1496497.79..."
3,angola,2011,4,-12.581454,13.397711,35,1.066994,POINT (1491426.344 -1411954.259),-12_5_13_39.tif,"POLYGON ((1488066.344 -1415314.259, 1494786.34..."
4,angola,2011,5,-12.578135,13.418748,37,1.750153,POINT (1493768.184 -1411575.617),-12_5_13_41.tif,"POLYGON ((1490408.184 -1414935.617, 1497128.18..."


In [11]:
# LOOPING OVER OCCURENCES
# WE BRUTEFORCE TEST ALL CHUNKS 
# OF A GIVEN COUNTRY,YEAR COMBINATION
for year in files:
    countries = files[year]
    for country in countries:
        occurences = gdf[gdf.year == year]
        occurences = occurences[occurences.country == country]
        for idx, row in occurences.iterrows():
            for file in countries[country]:
                tif = open_tif(file)
                tile = clip_geometry(row.geometry, tif)
                if tile is not None:
                    write_tile(
                        tile=tile,
                        name=str(row.geometry.x)+'_'+str(row.geometry.y)
                    )